<!-- cluster size ~ data count (cluster num reads num contacts num)

chr count pct < 5% 弃掉最少chr ~ data count 

cis pct > 10%  ~ data count

近距离互作 pair > 1000 ~ data count

弃掉trans? ~ data count

去掉背景spot -->


In [ ]:
import os
from filter_sprite import spot_qc,spot_filtered_Cs_infor_to_sprite
import yaml
import pandas as pd
from concurrent import futures
from functools import partial
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:

def spot_qc_and_filtered_Cs_infor_to_sprite(spot_id,out_dir,sample_id, spot_f_dir, min_C_size, 
                                max_C_size, min_chr_read_num_ratio,min_C_cis_ct_num_ratio):
    
    spot_infor, Cs_infor_fCsofcis, Cs_infor_fCsofcis=spot_qc(spot_id=spot_id,sample_id=sample_id, spot_f_dir=spot_f_dir, min_C_size=min_C_size, 
                                max_C_size=max_C_size, min_chr_read_num_ratio=min_chr_read_num_ratio,
                                min_C_cis_ct_num_ratio=min_C_cis_ct_num_ratio)
    out_sprite_dir=os.path.join(out_dir,f'clusters_{sample_id}_single_filtered_new_v2')
    os.makedirs(out_sprite_dir,exist_ok=True)
    out_sprite_f=os.path.join(out_sprite_dir,f'{spot_id}')
    spot_filtered_Cs_infor_to_sprite(Cs_infor_fCsofcis,out_sprite_f)
    return spot_infor, Cs_infor_fCsofcis, Cs_infor_fCsofcis

In [ ]:

def sample_spot_qc_and_filtered_Cs_infor_to_sprite(sample_id,out_dir,spot_f_dir,cpus,min_C_size,
                                                   max_C_size,min_chr_read_num_ratio,min_C_cis_ct_num_ratio):
    print(f'\n{sample_id}')
    out_final_spot_infor_f=os.path.join(out_dir,sample_id+'_spot_infor_final_v2.csv')
    if not os.path.exists(out_final_spot_infor_f):
        spot_infor_f=os.path.join('/home/spaceA/SpatialSPRITE_res',
                                sample_id+'_obs.csv')
        spot_infor=pd.read_csv(spot_infor_f,index_col=0)
        spotid_list=spot_infor.spotid.tolist()

        with futures.ProcessPoolExecutor(max_workers=cpus) as pool:
            func = partial(spot_qc_and_filtered_Cs_infor_to_sprite,out_dir=out_dir, sample_id=sample_id, spot_f_dir=spot_f_dir, min_C_size=min_C_size, 
                                    max_C_size=max_C_size, min_chr_read_num_ratio=min_chr_read_num_ratio,
                                    min_C_cis_ct_num_ratio=min_C_cis_ct_num_ratio)
            # results = list(tqdm(pool.map(func, spotid_list), total=len(spotid_list)))
            results = list(pool.map(func, spotid_list)) 
            allspot_infor, _, _ = zip(*results)
            del results
            del _
        out_spot_infor=pd.DataFrame()
        out_spot_infor['spot_id']=spotid_list
        out_spot_infor['sample']=sample_id
        columns = ['spot_raw_read_num', 'spot_raw_C_num', 'spot_raw_ct_num',
        'spot_fcsize_read_num', 'spot_fcsize_C_num', 'spot_fcsize_ct_num',
        'spot_fcsize_cis_ct_num', 'spot_fcsize_cis_ct_num_ratio',
        'spot_fcsizefchrrnp_read_num', 'spot_fcsizefchrrnp_ct_num',
        'spot_fcsizefchrrnp_cis_ct_num', 'spot_fcsizefchrrnp_cis_ct_num_ratio',
        'spot_fcsizefchrrnpfCsofcis_C_num', 'spot_fcsizefchrrnpfCsofcis_read_num',
        'spot_fcsizefchrrnpfCsofcis_ct_num', 'spot_fcsizefchrrnpfCsofcis_cis_ct_num',
        'spot_fcsizefchrrnpfCsofcis_cis_ct_num_ratio' ]
        for col in columns:
            out_spot_infor[col] = [x[col] for x in allspot_infor]
        out_spot_infor.to_csv(out_final_spot_infor_f,index=False) 

        print(f'{sample_id} is done!')

In [ ]:
out_dir='/home/spaceA/SpatialSPRITE_res/Filter_Spot_v4'
spot_f_dir='/home/spaceA/cluster'
min_C_size=2
max_C_size=1000
min_chr_read_num_ratio=0.15
min_C_cis_ct_num_ratio=0.2
cpus=50

os.makedirs(out_dir,exist_ok=True)
os.chdir(out_dir)

with open("/home/spaceA/config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

sampleid_list= config['spatial_infor'].keys()



In [ ]:
for sample_name in tqdm(sampleid_list):
    sample_spot_qc_and_filtered_Cs_infor_to_sprite(sample_id=sample_name,out_dir=out_dir,spot_f_dir=spot_f_dir,
                                                   cpus=cpus,min_C_size=min_C_size,
                                                   max_C_size=max_C_size,min_chr_read_num_ratio=min_chr_read_num_ratio,
                                                   min_C_cis_ct_num_ratio=min_C_cis_ct_num_ratio)

In [ ]:
spot_infor_all=pd.DataFrame()

for sample_name in sampleid_list:
    spot_infor_f=os.path.join(out_dir,sample_name+'_spot_infor_final_v2.csv')
    spot_infor_tmp=pd.read_csv(spot_infor_f,index_col=0)
    spot_infor_all=pd.concat([spot_infor_all,spot_infor_tmp])
    del spot_infor_tmp
spot_infor_all.head()


In [ ]:
spot_infor_all=spot_infor_all.reset_index()

raw_columns = [
    'spot_raw_read_num', 'spot_raw_C_num', 
    'spot_raw_ct_num','spot_fcsize_cis_ct_num',
    'spot_fcsize_cis_ct_num_ratio'
]
final_columns = [
    'spot_fcsizefchrrnpfCsofcis_read_num', 'spot_fcsizefchrrnpfCsofcis_C_num',
    'spot_fcsizefchrrnpfCsofcis_ct_num', 'spot_fcsizefchrrnpfCsofcis_cis_ct_num',
    'spot_fcsizefchrrnpfCsofcis_cis_ct_num_ratio'
]

df_raw = spot_infor_all[['sample','spot_id'] + raw_columns].copy()
df_raw['group'] = 'raw'
df_raw = df_raw.rename(columns={
    'spot_raw_read_num': 'read_num',
    'spot_raw_C_num': 'C_num',
    'spot_raw_ct_num': 'ct_num',
   'spot_fcsize_cis_ct_num': 'cis_ct_num',
   'spot_fcsize_cis_ct_num_ratio': 'cis_ct_num_ratio'
})


df_final = spot_infor_all[['sample','spot_id'] + final_columns].copy()
df_final['group'] = 'final'
df_final = df_final.rename(columns={
    'spot_fcsizefchrrnpfCsofcis_read_num': 'read_num',
    'spot_fcsizefchrrnpfCsofcis_C_num': 'C_num',
    'spot_fcsizefchrrnpfCsofcis_ct_num': 'ct_num',
    'spot_fcsizefchrrnpfCsofcis_cis_ct_num': 'cis_ct_num',
    'spot_fcsizefchrrnpfCsofcis_cis_ct_num_ratio': 'cis_ct_num_ratio'
})

result_df = pd.concat([df_raw, df_final], ignore_index=True)


result_df = result_df[['sample', 'spot_id', 'group', 'read_num', 'C_num', 'ct_num', 'cis_ct_num', 'cis_ct_num_ratio']]

result_df.head()

In [ ]:
result_df.to_csv('all_samples_raw_and_filter_v2_spot_infor_final.csv')

In [ ]:
plt.figure(figsize=(10,8))

sns.barplot(x='sample', y='read_num', hue='group',
            data=result_df)
plt.xticks(rotation=45);
plt.savefig('allsamples_read_num_raw_final_v2.png')

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(x='sample', y='cis_ct_num_ratio', hue='group',
            data=result_df)
plt.xticks(rotation=45);
plt.savefig('allsamples_cis_ct_num_ratio_raw_final_v2.png')